In [1]:
import requests
import time

headers = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
           'Accept-Encoding': 'gzip, deflate, br',
           'Accept-Language': 'zh-CN,zh;q=0.9',
           'Cache-Control': 'no-cache',
           'Connection': 'keep-alive',
           'User-Agent': 'Mozilla/6.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}

proxies = {
    "http": "http://125.123.157.22:3000",
    "https": "https://127.0.0.1:8080",
}

#print(requests.validate(proxies))

def get_page(url):
    time.sleep(10)
    try:
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'
        print(resp)
        #print(resp.text)
        return resp
    except requests.exceptions.ConnectionError as e:
        print(e)

### 设置代理IP

In [2]:
import urllib

enable_proxy = True
proxy_handler = urllib.request.ProxyHandler({"http":'http://www.66ip.cn/'})
null_proxy_handler = urllib.request.ProxyHandler({})
if enable_proxy:
    opener = urllib.request.build_opener(proxy_handler)
else:
    opener = urllib.request.build_opener(null_proxy_handler)

In [3]:
bili = "https://www.bilibili.com"

s = requests.session()
s.keep_alive = False
bpage = get_page(bili)

<Response [200]>


In [4]:
import bs4
import re
from bs4 import BeautifulSoup

def get_all_href(soup):
    a_list = soup.find_all('a')
    url_list = []
    for a in a_list:
        if a.attrs.get('href'):
            #print(a.attrs.get('href'))
            url_list.append(a.attrs.get('href'))
    return url_list

#urls = get_all_href(soup)

def get_danmu(url):
    #获取bv号
    pattern = re.compile(r'BV([\w]*)')
    bv = "BV" + re.findall(pattern, url)[0]
    print(bv)
    cid = get_cid(bv)
    if cid is None:
        return None
    dresp = get_danmu_xml(cid)
    soup = BeautifulSoup(dresp.text)
    danmu = soup.find_all('d')
    return danmu
    #write...
    
def get_cid(bv):
    url = "https://api.bilibili.com/x/player/pagelist?bvid=" + bv + "&jsonp=jsonp"
    resp = get_page(url)
    if resp is None:
        return None
    csoup = BeautifulSoup(resp.text)
    info = eval(str(csoup.string))
    cid = info.get('data')[0].get('cid')
    return cid
    
def get_danmu_xml(cid):
    url = "https://api.bilibili.com/x/v1/dm/list.so?oid="+str(cid)
    #danmu_path = "http://comment.bilibili.com/"+str(cid)+".xml"
    resp = get_page(url)
    return resp

def get_danmu_bytes(bv):
    cid = get_cid(bv)
    contents = get_danmu_xml(cid)
    return contents

In [5]:
s = requests.session()
s.keep_alive = False

#d_list = get_danmu("https://www.bilibili.com/video/BV1p5411s7fF")

In [5]:
def get_d_contents(danmu):
    dm = []
    for d in danmu:
        for c in d.contents:
            #print(isinstance(c, bs4.element.Tag))
            if(isinstance(c, bs4.element.NavigableString)):
                dm.append(c.strip())
    return dm

In [20]:
import os
import time
import pandas as pd

'''
def set_proxies():
    enable_proxy = True
    proxy_handler = urllib.request.ProxyHandler({"http":'http://www.66ip.cn/areaindex_19/1.html'})
    null_proxy_handler = urllib.request.ProxyHandler({})
    if enable_proxy:
        opener = urllib.request.build_opener(proxy_handler)
    else:
        opener = urllib.request.build_opener(null_proxy_handler)
'''

# root：爬虫起点url
# path：抓取的弹幕文件存储位置
def crawl(root, path, maximum=100):
    urls = [root]
    # path = 'Jerry'
    visited = []
    count = 0
    if path not in os.listdir():
        os.makedirs(path)
    while len(urls)>0 and count<maximum:
        #读取url
        url = urls.pop(0)
        
        #已访问：跳过本次循环
        if url in visited:
            continue
        # set_proxies()
        _html = get_page(url)
        #访问出错，跳过本次循环
        if _html is None:
            time.sleep(10)
            continue
        
        _soup = BeautifulSoup(_html.text)
        _html_p = _soup.prettify()
        if _soup.title:
            url_name = _soup.title.contents[0]
        else:
            continue
            #url_name = 'nt'+ str(count)
        
        #获取下一层url
        related = _soup.findAll(name="div", attrs={"class" :"recommend-list"})
        print(related)
        rsoup = BeautifulSoup(str(related[0]))
        r_list = rsoup.findAll(name="a", attrs={"class": "title"})
        print(r_list)
        for r in r_list:
            href = r.attrs.get('href')
            urls.append("https://www.bilibili.com" + href)
        
        #将当前url写入已遍历的visited数组中
        visited.append(url)
        
        
        #在相应文件目录下写入.html文件
        # set_proxies()
        danmu = get_danmu(url)
        if danmu is None:
            time.sleep(10)
            continue
        danmu = get_d_contents(danmu)
        df = pd.DataFrame(danmu, columns=['danmu'])
        
        #print(df)
        csv = os.path.join(path, url_name + '.csv')
        try:
            df.to_csv(csv, index=False, sep=',', encoding="utf_8_sig")
        except Exception as e:
            print(e)
        count += 1

In [21]:
crawl("https://www.bilibili.com/video/BV1aJ411C7tb", 'digital', maximum=80)

<Response [200]>
[<div class="recommend-list report-wrap-module report-scroll-module" id="reco_list"><!-- --><div><div class="loading-card"><div class="cover"></div><div class="info"><p class="title"></p><p class="up"></p><p class="desc"></p></div></div><div class="loading-card"><div class="cover"></div><div class="info"><p class="title"></p><p class="up"></p><p class="desc"></p></div></div><div class="loading-card"><div class="cover"></div><div class="info"><p class="title"></p><p class="up"></p><p class="desc"></p></div></div><div class="loading-card"><div class="cover"></div><div class="info"><p class="title"></p><p class="up"></p><p class="desc"></p></div></div><div class="loading-card"><div class="cover"></div><div class="info"><p class="title"></p><p class="up"></p><p class="desc"></p></div></div><div class="loading-card"><div class="cover"></div><div class="info"><p class="title"></p><p class="up"></p><p class="desc"></p></div></div><div class="loading-card"><div class="cover"><

In [69]:
# print(series_list[0].to_frame().T)

     了    的    我    ，    吃  万州   是   吧   ？   。  ...  油太宽  女儿  别说  断  上新菜  摸  \
0  276  171  162  160  121  96  84  80  79  73  ...    1   1   1  1    1  1   

   不齐  试试  方程式  68  
0   1   1    1   1  

[1 rows x 1289 columns]
